In [1]:
from datetime import datetime

In [2]:
import json
import re
import requests

In [3]:
access_token = '210555813636350|rKIeB30Wq0TWA-a0XoIC8OKBrpo'

In [ ]:
https://www.facebook.com/ads/library/?active_status=all&ad_type=all&country=US&impression_search_field=has_impressions_lifetime&view_all_page_id=153080620724

In [4]:
def get_ad_archive_id(data):
    """
    Extract ad_archive_id from ad_snapshot_url
    """
    return re.search(r"/\?id=([0-9]+)", data["ad_snapshot_url"]).group(1)


class FbAdsLibraryTraversal:
    default_url_pattern = (
        "https://graph.facebook.com/{}/ads_archive?access_token={}&"
        + "fields={}&search_terms={}&ad_reached_countries={}&search_page_ids={}&"
        + "ad_active_status={}&limit={}"
    )
    default_api_version = "v4.0"

    def __init__(
        self,
        access_token,
        fields,
        search_term,
        country,
        search_page_ids="",
        ad_active_status="active",
        after_date="1970-01-01",
        page_limit=500,
        api_version=None,
        retry_limit=3,
    ):
        self.page_count = 0
        self.access_token = access_token
        self.fields = fields
        self.search_term = search_term
        self.country = country
        self.after_date = after_date
        self.search_page_ids = search_page_ids
        self.ad_active_status = ad_active_status
        self.page_limit = page_limit
        self.retry_limit = retry_limit
        if api_version is None:
            self.api_version = self.default_api_version
        else:
            self.api_version = api_version

    def generate_ad_archives(self):
        next_page_url = self.default_url_pattern.format(
            self.api_version,
            self.access_token,
            self.fields,
            self.search_term,
            self.country,
            self.search_page_ids,
            self.ad_active_status,
            self.page_limit,
        )
        return self.__class__._get_ad_archives_from_url(
            next_page_url, after_date=self.after_date, retry_limit=self.retry_limit
        )

    @staticmethod
    def _get_ad_archives_from_url(
        next_page_url, after_date="1970-01-01", retry_limit=3
    ):
        last_error_url = None
        last_retry_count = 0
        start_time_cutoff_after = datetime.strptime(
            after_date, "%Y-%m-%d"
        ).timestamp()

        while next_page_url is not None:
            response = requests.get(next_page_url)
            response_data = json.loads(response.text)
            if "error" in response_data:
                if next_page_url == last_error_url:
                    # failed again
                    if last_retry_count >= retry_limit:
                        raise Exception(
                            "Error message: [{}], failed on URL: [{}]".format(
                                json.dumps(response_data["error"]), next_page_url
                            )
                        )
                else:
                    last_error_url = next_page_url
                    last_retry_count = 0
                last_retry_count += 1
                continue

            filtered = list(
                filter(
                    lambda ad_archive: datetime.strptime(
                        ad_archive["ad_delivery_start_time"], "%Y-%m-%dT%H:%M:%S%z"
                    ).timestamp()
                    >= start_time_cutoff_after,
                    response_data["data"],
                )
            )
            if len(filtered) == 0:
                # if no data after the after_date, break
                next_page_url = None
                break
            yield filtered

            if "paging" in response_data:
                next_page_url = response_data["paging"]["next"]
            else:
                next_page_url = None

    @classmethod
    def generate_ad_archives_from_url(cls, failure_url, after_date="1970-01-01"):
        """
        if we failed from error, later we can just continue from the last failure url
        """
        return cls._get_ad_archives_from_url(failure_url, after_date=after_date)


In [14]:
fb_ads = FbAdsLibraryTraversal(access_token, 'all','Trump', 'United States')

In [15]:
generator = fb_ads.generate_ad_archives()

In [19]:
generator.send()

TypeError: send() takes exactly one argument (0 given)

In [12]:
def count_ads(generator_ad_archives,is_verbose=False):
    """
    Count how many ad_archives match your query
    """
    count = 0
    for ad_archives in generator_ad_archives:
        count += len(ad_archives)
        if is_verbose:
            print("counting %d" % count)
    print("Total number of ads match the query: {}".format(count))

In [13]:
count_ads(generator)

OSError: [Errno 22] Invalid argument

In [ ]:
import yaml
import requests
import csv
import re
from tqdm import tqdm
from itertools import product

with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

assert config['search_total'] % config['page_total'] == 0, \
    "search_total should be a multiple of page_total."

params = {
    'access_token': config['access_token'],
    'ad_type': 'POLITICAL_AND_ISSUE_ADS',
    'ad_reached_countries': "['US']",
    'ad_active_status': config['ad_active_status'],
    'search_terms': config.get('search_terms'),
    'search_page_ids': ",".join(config.get('search_page_ids', [])),
    'fields': ",".join(config['query_fields']),
    'limit': config['page_total']
}

REGIONS = set(config['regions'])
DEMOS = set(product(config['demo_ages'], config['demo_genders']))

f1 = open('fb_ads.csv', 'w')
w1 = csv.DictWriter(f1, fieldnames=config['output_fields'],
                    extrasaction='ignore')
w1.writeheader()

f2 = open('fb_ads_demos.csv', 'w')
w2 = csv.DictWriter(f2, fieldnames=config['demo_fields'],
                    extrasaction='ignore')
w2.writeheader()

f3 = open('fb_ads_regions.csv', 'w')
w3 = csv.DictWriter(f3, fieldnames=config['region_fields'],
                    extrasaction='ignore')
w3.writeheader()

pbar = tqdm(total=config['search_total'], smoothing=0)

for _ in range(int(config['search_total'] / config['page_total'])):
    r = requests.get('https://graph.facebook.com/v5.0/ads_archive',
                     params=params)
    data = r.json()
    for ad in data['data']:
        # The ad_id is encoded in the ad snapshot URL
        # and cannot be accessed as a normal field. (?!?!)

        ad_id = re.search(r'\d+', ad['ad_snapshot_url']).group(0)
        ad_url = 'https://www.facebook.com/ads/library/?id=' + ad_id

        # write to the unnested files
        demo_set = set()
        for demo in ad['demographic_distribution']:
            demo.update({'ad_id': ad_id})
            w2.writerow(demo)
            demo_set.add((demo['age'], demo['gender']))

        # Impute a percentage of 0
        # for demos with insufficient data
        unused_demos = DEMOS - demo_set
        for demo in unused_demos:
            w2.writerow({
                'ad_id': ad_id,
                'age': demo[0],
                'gender': demo[1],
                'percentage': 0
            })

        region_set = set()
        for region in ad['region_distribution']:
            region.update({'ad_id': ad_id})
            w3.writerow(region)
            region_set.add(region['region'])

        # Impute a percentage of 0
        # for states with insufficient data
        unused_regions = REGIONS - region_set
        for region in unused_regions:
            w3.writerow({
                'ad_id': ad_id,
                'region': region,
                'percentage': 0
            })

        ad.update({'ad_id': ad_id,
                   'ad_url': ad_url,
                   'impressions_min': ad['impressions']['lower_bound'],
                   'impressions_max': ad['impressions']['upper_bound'],
                   'spend_min': ad['spend']['lower_bound'],
                   'spend_max': ad['spend']['upper_bound'],
                   })

        w1.writerow(ad)
        pbar.update()

    # if we have scraped all the ads, exit
    if 'paging' not in data:
        break

    params.update({'after': data['paging']['cursors']['after']})

f1.close()
f2.close()
f3.close()
pbar.close()